In [2]:
import os
from esm.models.esmc import ESMC
from esm.sdk.api import ESMProtein, LogitsConfig
import torch
import math
import tqdm

In [3]:
import_data = "../Data/Protein_Gym_Datasets/GFP_AEQVI_Sarkisyan_2016.csv"
data_set = import_data.split("/").split(".")[0]
#HIS7_YEAST_Pokusaeva_2019
input_data = []
with open(import_data, "r") as infile:
    for line in infile.readlines():
        input_data.append(line[:-1].split(","))

to_encode = [line[1] for line in input_data[1:]]

In [5]:
import gc
gc.collect()
torch.cuda.empty_cache()

for method in ["esmc_300m","esmc_600m"]:
    pad = 1 + math.floor(math.log(len(to_encode), 10))
    outfile = "../Data/ESM_Embeddings"
    with tqdm.tqdm(total=len(to_encode)) as pbar:
        for i, sequence in enumerate(to_encode):
            protein = ESMProtein(sequence=sequence)
            client = ESMC.from_pretrained(method).to("cuda")  # or "cpu"
            protein_tensor = client.encode(protein)
            logits_output = client.logits(protein_tensor,
                                        LogitsConfig(sequence=True, return_embeddings=True))

            residue_embeddings = logits_output.embeddings[0, 1:-1, :]
            protein_embedding = residue_embeddings.mean(dim=0)
            outfile = f"{i:0{pad}d}"
            torch.save(protein_embedding, f"../Data/Embeddings/{data_set}/{method}/{outfile}", _use_new_zipfile_serialization=False)
            pbar.update(1)

  0%|          | 0/496137 [00:01<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 7.74 GiB of which 26.56 MiB is free. Including non-PyTorch memory, this process has 7.36 GiB memory in use. Of the allocated memory 6.65 GiB is allocated by PyTorch, and 579.90 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)